In [1]:
import os;import sys
import pandas as pd;import numpy as np;from itertools import combinations, permutations, product
from datetime import datetime;from datetime import date
from collections import Counter;import matplotlib ; from matplotlib import pyplot as plt
import networkx as nx; from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,AutoMinorLocator)
from scipy.io import loadmat
from scipy.sparse import csr_matrix
from scipy import stats
from sknetwork.clustering import Louvain, BiLouvain, modularity, bimodularity
import nbimporter ; import importlib as imp ; from scipy.optimize import minimize 
from scipy.optimize import LinearConstraint
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score
from sklearn.metrics import (confusion_matrix,  
                           accuracy_score) 
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from random import gauss, random, choices
import pickle; import math
import geopandas as gpd
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant
from scipy.stats.distributions import chi2


In [2]:
import statsmodels.api as sm 

In [3]:
path=os.getcwd()

In [4]:
def flatten(L):
    return [l for ll in L for l in ll]

In [5]:
def partition(l, nbin):
    dw = int(np.ceil(len(l)/nbin))
    ll = []
    i0 = 0
    while i0+dw<len(l):
        ll.append(l[i0:i0+dw])
        i0+=dw
    if i0 < len(l):
        ll[-1] = ll[-1]+l[i0:]
    return ll

In [6]:
def getSig(p):
    if p<0.005: return "***"
    elif p<0.01: return "**"
    elif p<0.05: return "*"
    else: return " "

<h2 style='text-align: center;'> Import Model Data </h2>

In [27]:
data = pd.read_csv('./../indicators/model_data_bonds_loans_d0_df.csv')
data = data[(data.Corporate==1)]
data['FCY']=data.LCY.apply(lambda x: 1 if x==0 else 0)

In [28]:
data

,Quarter,Year,Country,Delta_Interest_EU,Delta_Interest_JPY,Delta_Interest_US,ECB,Exchange_rate,FFR,FX_long,...,Amount,LCY,Currency,Domestic,Corporate,Industry,Bond,Tenor,Tenor_bracket,FCY
0,4,2020,Malaysia,3.8046,3.7546,3.9646,0.25,4.147800,0.09,0.016301,...,0.0247,1,MYR,1,1,Consumer staples,0,9.404125,5-10 yrs,0
1,2,2010,Thailand,-0.5000,0.9500,1.0500,1.75,32.491300,0.20,0.018714,...,0.0587,1,THB,1,1,Energy,0,18.000000,>10 yrs,0
2,3,2022,Mexico,7.7200,8.9200,7.5400,1.50,20.010100,1.68,0.033409,...,0.1365,1,MXN,1,1,Industrials,1,4.000000,0-5 yrs,0
5,3,2011,South Africa,3.2400,5.1900,5.4200,2.25,6.785900,0.07,0.041921,...,0.3650,0,USD,1,1,Utilities,0,9.404125,5-10 yrs,1
6,2,2011,Turkey,-0.5000,1.2000,1.4000,2.00,1.572213,0.10,0.040756,...,0.0682,0,USD,1,1,Utilities,0,11.000000,>10 yrs,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2233,1,2024,Indonesia,2.1800,6.6300,1.6000,4.75,15610.818640,5.33,0.026560,...,0.0004,1,IDR,1,1,Materials,1,5.000000,0-5 yrs,0
2234,1,2023,Malaysia,0.7140,4.1640,-0.1860,3.75,4.321800,4.65,0.013289,...,0.0002,1,MYR,1,1,Industrials,1,9.404125,5-10 yrs,0
2235,1,2023,Malaysia,0.7140,4.1640,-0.1860,3.75,4.321800,4.65,0.013289,...,0.0002,1,MYR,1,1,Industrials,1,1.000000,0-5 yrs,0
2236,4,2023,Indonesia,2.2000,6.6500,1.6200,4.75,15526.000000,5.33,0.027528,...,0.0002,1,IDR,1,1,Materials,1,5.000000,0-5 yrs,0


In [22]:
data1=data.sample(frac=1)
data2=data.sample(frac=1)
data3=data.sample(frac=1)
data4=data.sample(frac=1)
data5=data.sample(frac=1)
data6=data.sample(frac=1)

In [24]:
data1.Country

1111          Brazil
1070       Indonesia
43            Mexico
258     South Africa
900            India
            ...     
1320           India
448            India
209            India
897           Brazil
1491        Malaysia
Name: Country, Length: 1534, dtype: object

In [29]:
data['Tenor']=data1.Tenor.values
data['Amount']=data2.Amount.values
data['Country']=data3.Country.values
data['Bond']=data4.Bond.values
data['Currency']=data5.Currency.values
data['FCY']=data6.FCY.values

In [31]:
data.to_csv('./toy_model_data.csv',index=False)

<h2 style='text-align: center;'> Supplementary Table: + Malaysia & Turkey </h2>

In [9]:
countries = ['India',
             'Mexico',
             'Brazil',
             'South Africa',
             'Indonesia',
             'Thailand',
             'Malaysia',
             'Turkey'
            ]

In [10]:
mdl = data.copy();
mdl = mdl[mdl.Country.apply(lambda x: x in countries)]
len(mdl)

1534

In [11]:
mdl.keys()

Index(['Quarter', 'Year', 'Country', 'Delta_Interest_EU', 'Delta_Interest_JPY',
       'Delta_Interest_US', 'ECB', 'Exchange_rate', 'FFR', 'FX_long',
       'FX_short', 'Fin_depth', 'Fin_dev', 'GDP Growth', 'GDP per cap',
       'Interest rate', 'JPY', 'Pol_Risk', 'Ratings FC', 'Ratings LC', 'VIX',
       'WUI', 'market_volume_bonds', 'market_volume_loans', 'Amount', 'LCY',
       'Currency', 'Domestic', 'Corporate', 'Industry', 'Bond', 'Tenor',
       'Tenor_bracket', 'FCY'],
      dtype='object')

In [12]:
# Create a DataFrame with the one-hot encoded columns

# Drop first column to avoid the dummy variable trap
# ------ One-Hot Country encoding --------------- #
one_hot_df = pd.get_dummies(mdl.Country, drop_first=True)
mdl = pd.concat([mdl, one_hot_df], axis=1)
FE_country_ = ['India',
              'Mexico',
              'South Africa',
              'Indonesia',
              'Thailand',
               'Malaysia',
               'Turkey'
              ]

# ------ One-Hot Tenor encoding --------------- #
one_hot_df = pd.get_dummies(mdl.Tenor_bracket, drop_first = True)
one_hot_df=one_hot_df.rename(columns = {'0-5 yrs':'tenor_1',
                    '5-10 yrs': 'tenor_2',
                    '>10 yrs': 'tenor_3',
                             } )
mdl = pd.concat([mdl, one_hot_df], axis=1)
FE_tenor_ = ['tenor_{}'.format(i) for i in [1,2,3]] 

#-------- TRANSFORM VARIABLES ------------#
mdl['Amount'] = mdl['Amount'].apply(lambda x: np.log(x))
mdl['WUI'] = mdl['WUI'].apply(lambda x: x/1000)

In [13]:
### Table for main
vars_cntrl_ = ['Amount','Domestic','Bond',
              'GDP Growth','Tenor','GDP per cap']

vars_list_ = [
        ['FX_short','Ratings FC','Pol_Risk','VIX'] + vars_cntrl_ + FE_country_,
        ['FX_short','Pol_Risk','VIX'] + vars_cntrl_ + FE_country_,
        ]

all_vars_ = ['FX_short','Ratings FC','Pol_Risk','VIX'] + \
             vars_cntrl_

log_reg = {}
for i,vars_ in enumerate(vars_list_):
    log_reg[i] = \
     sm.Logit(mdl.FCY, mdl[vars_].to_numpy()).fit(maxiter=1000)
    
df_results = pd.DataFrame(
                columns=[i for i,v in enumerate(vars_list_)],
                index = all_vars_
                        )

for i, vars_ in enumerate(vars_list_):
    for j,v in enumerate(vars_):
        if v in all_vars_:
            df_results.loc[v][i] = '{:.2f}{}({:.2f})'.format(
                            log_reg[i].params[j],
                            getSig(log_reg[i].pvalues[j]),
                            #log_reg[i].pvalues[j],
                            log_reg[i].bse[j])

accuracy = []
for i,lr in log_reg.items():
    X_train, X_test, y_train, y_test = train_test_split(
                    mdl[vars_list_[i]],
                    mdl.FCY,test_size=0.33, random_state=20)

    y_prob = lr.predict(X_test)
    y_pred = [0 if p<0.5 else 1 for p in y_prob]

    # model accuracy versus guess major class
    accuracy.append(metrics.accuracy_score(y_test.values, y_pred))

df_results.loc['Observations'] = [len(mdl)]*len(vars_list_)
df_results.loc['McFaddens R-squared'] = \
                ['{:.2f}'.format(log_reg[i].prsquared) 
                for i in log_reg.keys()]

df_results.loc['Accuracy'] = \
                ['{:.1f}'.format(100*a) for a in accuracy]

df_results=df_results.rename(columns={i:'Reg {}'.format(i+1)
                          for i in log_reg.keys()})

np.set_printoptions(linewidth=110)
df_results=df_results.rename(index= {'FX_short':'FX_RISK',
                           'Ratings FC': 'SOV_RISK',
                           'Pol_Risk':'POL_RISK',
                           'Amount':'AMOUNT',
                           'Domestic':'DOMESTIC',
                           'Bond':'BOND',
                           'GDP Growth': 'GDP_GROWTH',
                           'Tenor':'TENOR'})
df_results.transpose()

Optimization terminated successfully.
         Current function value: 0.283973
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.290554
         Iterations 8


,FX_RISK,SOV_RISK,POL_RISK,VIX,AMOUNT,DOMESTIC,BOND,GDP_GROWTH,TENOR,GDP per cap,Observations,McFaddens R-squared,Accuracy
Reg 1,74.39***(11.28),0.40***(0.09),-0.01 (0.38),0.04**(0.01),0.68***(0.07),-5.55***(0.68),-2.05***(0.25),0.03 (0.03),-0.09***(0.02),-0.00 (0.00),1534,0.51,89.3
Reg 2,90.60***(10.93),NaN,-0.12 (0.37),0.04***(0.01),0.66***(0.07),-5.01***(0.57),-2.12***(0.25),0.03 (0.03),-0.07***(0.01),0.00***(0.00),1534,0.50,89.5


In [93]:
df_results.to_csv('./../results/Table_S_Malaysia_Turkey.csv')

<h2 style='text-align: center;'> Supplementary Table: variable alternatives </h2>

In [94]:
countries = ['India',
             'Mexico',
             'Brazil',
             'South Africa',
             'Indonesia',
             'Thailand',
             #'Malaysia',
             #'Turkey'
            ]

In [95]:
mdl = data.copy();
mdl = mdl[mdl.Country.apply(lambda x: x in countries)]
len(mdl)

1112

In [96]:
mdl.keys()

Index(['Quarter', 'Year', 'Country', 'Delta_Interest_EU', 'Delta_Interest_JPY',
       'Delta_Interest_US', 'ECB', 'Exchange_rate', 'FFR', 'FX_long',
       'FX_short', 'Fin_depth', 'Fin_dev', 'GDP Growth', 'GDP per cap',
       'Interest rate', 'JPY', 'Pol_Risk', 'Ratings FC', 'Ratings LC', 'VIX',
       'WUI', 'market_volume_bonds', 'market_volume_loans', 'Amount', 'LCY',
       'Currency', 'Domestic', 'Corporate', 'Industry', 'Bond', 'Tenor',
       'Tenor_bracket', 'FCY'],
      dtype='object')

In [97]:
# Create a DataFrame with the one-hot encoded columns

# Drop first column to avoid the dummy variable trap
# ------ One-Hot Country encoding --------------- #
one_hot_df = pd.get_dummies(mdl.Country, drop_first=True)
mdl = pd.concat([mdl, one_hot_df], axis=1)
FE_country_ = ['India',
              'Mexico',
              'South Africa',
              'Indonesia',
              'Thailand',
               #'Malaysia',
               #'Turkey'
              ]

# ------ One-Hot Tenor encoding --------------- #
one_hot_df = pd.get_dummies(mdl.Tenor_bracket, drop_first = True)
one_hot_df=one_hot_df.rename(columns = {'0-5 yrs':'tenor_1',
                    '5-10 yrs': 'tenor_2',
                    '>10 yrs': 'tenor_3',
                             } )
mdl = pd.concat([mdl, one_hot_df], axis=1)
FE_tenor_ = ['tenor_{}'.format(i) for i in [1,2,3]] 

#-------- TRANSFORM VARIABLES ------------#
mdl['Amount'] = mdl['Amount'].apply(lambda x: np.log(x))
mdl['WUI'] = mdl['WUI'].apply(lambda x: x/1000)

In [99]:
### Table for main
vars_cntrl_ = ['Amount','Domestic','Bond',
              'GDP Growth','Tenor']

vars_list_ = [
        ['FX_short','Ratings FC','Pol_Risk','VIX'] + vars_cntrl_ + FE_country_,
        ['FX_short','Ratings FC','Pol_Risk','WUI'] + vars_cntrl_ + FE_country_,
         ['FX_long','Ratings FC','Pol_Risk','VIX'] + vars_cntrl_ + FE_country_,
        ['FX_long','Ratings FC','Pol_Risk','WUI'] + vars_cntrl_ + FE_country_,]

all_vars_ = ['FX_short','FX_long','Ratings FC','Pol_Risk','VIX','WUI'] + \
             vars_cntrl_

log_reg = {}
for i,vars_ in enumerate(vars_list_):
    log_reg[i] = \
     sm.Logit(mdl.FCY, mdl[vars_].to_numpy()).fit(maxiter=1000)
    
df_results = pd.DataFrame(
                columns=[i for i,v in enumerate(vars_list_)],
                index = all_vars_
                        )

for i, vars_ in enumerate(vars_list_):
    for j,v in enumerate(vars_):
        if v in all_vars_:
            df_results.loc[v][i] = '{:.2f}{}({:.2f})'.format(
                            log_reg[i].params[j],
                            getSig(log_reg[i].pvalues[j]),
                            #log_reg[i].pvalues[j],
                            log_reg[i].bse[j])

accuracy = []
for i,lr in log_reg.items():
    X_train, X_test, y_train, y_test = train_test_split(
                    mdl[vars_list_[i]],
                    mdl.FCY,test_size=0.33, random_state=20)

    y_prob = lr.predict(X_test)
    y_pred = [0 if p<0.5 else 1 for p in y_prob]

    # model accuracy versus guess major class
    accuracy.append(metrics.accuracy_score(y_test.values, y_pred))

df_results.loc['Observations'] = [len(mdl)]*len(vars_list_)
df_results.loc['McFaddens R-squared'] = \
                ['{:.2f}'.format(log_reg[i].prsquared) 
                for i in log_reg.keys()]

df_results.loc['Accuracy'] = \
                ['{:.1f}'.format(100*a) for a in accuracy]

df_results=df_results.rename(columns={i:'Reg {}'.format(i+1)
                          for i in log_reg.keys()})

np.set_printoptions(linewidth=110)
df_results=df_results.rename(index= {'FX_short':'FX_RISK_SHORT',
                                     'FX_long':'FX_RISK_LONG',
                           'Ratings FC': 'SOV_RISK',
                           'Pol_Risk':'POL_RISK',
                           'Amount':'AMOUNT',
                           'Domestic':'DOMESTIC',
                           'Bond':'BOND',
                           'GDP Growth': 'GDP_GROWTH',
                           'Tenor':'TENOR'})
df_results.transpose()

Optimization terminated successfully.
         Current function value: 0.352281
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.355301
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.373328
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.376121
         Iterations 8


,FX_RISK_SHORT,FX_RISK_LONG,SOV_RISK,POL_RISK,VIX,WUI,AMOUNT,DOMESTIC,BOND,GDP_GROWTH,TENOR,Observations,McFaddens R-squared,Accuracy
Reg 1,78.69***(12.54),NaN,0.33***(0.08),0.14 (0.41),0.04***(0.01),NaN,0.69***(0.07),-5.47***(0.69),-1.93***(0.25),0.03 (0.03),-0.09***(0.02),1112,0.41,84.7
Reg 2,77.40***(12.08),NaN,0.32***(0.08),-0.03 (0.41),NaN,0.04 (0.02),0.69***(0.07),-5.30***(0.67),-1.99***(0.26),0.03 (0.03),-0.09***(0.02),1112,0.41,85.6
Reg 3,NaN,20.70 (22.44),0.48***(0.08),-0.28 (0.44),0.04***(0.01),NaN,0.64***(0.07),-5.13***(0.63),-1.97***(0.25),-0.00 (0.03),-0.10***(0.02),1112,0.38,86.1
Reg 4,NaN,34.29 (21.77),0.45***(0.08),-0.35 (0.44),NaN,0.03 (0.02),0.64***(0.07),-5.14***(0.63),-2.05***(0.26),-0.01 (0.03),-0.10***(0.02),1112,0.37,86.6


In [83]:
df_results.to_csv('./../results/Table_S_variable_alternatives.csv')

<h2 style='text-align: center;'> Supplementary Table: Separate Bonds and Loans </h2>

In [119]:
countries = ['India',
             'Mexico',
             'Brazil',
             'South Africa',
             'Indonesia',
             'Thailand',
            # 'Malaysia',
            # 'Turkey'
            ]

In [120]:
mdl = data.copy();
mdl = mdl[mdl.Country.apply(lambda x: x in countries)]
len(mdl)

1112

In [121]:
mdl.keys()

Index(['Quarter', 'Year', 'Country', 'Delta_Interest_EU', 'Delta_Interest_JPY',
       'Delta_Interest_US', 'ECB', 'Exchange_rate', 'FFR', 'FX_long',
       'FX_short', 'Fin_depth', 'Fin_dev', 'GDP Growth', 'GDP per cap',
       'Interest rate', 'JPY', 'Pol_Risk', 'Ratings FC', 'Ratings LC', 'VIX',
       'WUI', 'market_volume_bonds', 'market_volume_loans', 'Amount', 'LCY',
       'Currency', 'Domestic', 'Corporate', 'Industry', 'Bond', 'Tenor',
       'Tenor_bracket', 'FCY'],
      dtype='object')

In [122]:
# Create a DataFrame with the one-hot encoded columns

# Drop first column to avoid the dummy variable trap
# ------ One-Hot Country encoding --------------- #
one_hot_df = pd.get_dummies(mdl.Country, drop_first=True)
mdl = pd.concat([mdl, one_hot_df], axis=1)
FE_country_ = ['India',
              'Mexico',
              'South Africa',
              'Indonesia',
              'Thailand',
               #'Malaysia',
              # 'Turkey'
              ]

# ------ One-Hot Tenor encoding --------------- #
one_hot_df = pd.get_dummies(mdl.Tenor_bracket, drop_first = True)
one_hot_df=one_hot_df.rename(columns = {'0-5 yrs':'tenor_1',
                    '5-10 yrs': 'tenor_2',
                    '>10 yrs': 'tenor_3',
                             } )
mdl = pd.concat([mdl, one_hot_df], axis=1)
FE_tenor_ = ['tenor_{}'.format(i) for i in [1,2,3]] 

#-------- TRANSFORM VARIABLES ------------#
mdl['Amount'] = mdl['Amount'].apply(lambda x: np.log(x))
mdl['WUI'] = mdl['WUI'].apply(lambda x: x/1000)

In [131]:
### Table for main
vars_cntrl_ = ['Amount','Domestic',
              'GDP Growth','Tenor']

vars_ = ['FX_short','Ratings FC','Pol_Risk','VIX'] + vars_cntrl_ + FE_country_

all_vars_ = ['FX_short','Ratings FC','Pol_Risk','VIX'] + \
             vars_cntrl_

log_reg = {}

for b in [0,1]:
    log_reg[b] = \
     sm.Logit(mdl[mdl.Bond==b].FCY, 
              mdl[mdl.Bond==b][vars_].to_numpy()).fit(maxiter=5000,
                                                     method='minimize')
    
df_results = pd.DataFrame(
                columns=[0,1],
                index = all_vars_
                        )

for i in [0,1]:
    for j,v in enumerate(vars_):
        if v in all_vars_:
            df_results.loc[v][i] = '{:.2f}{}({:.2f})'.format(
                            log_reg[i].params[j],
                            getSig(log_reg[i].pvalues[j]),
                            #log_reg[i].pvalues[j],
                            log_reg[i].bse[j])

accuracy = []
for b,lr in log_reg.items():
    X_train, X_test, y_train, y_test = train_test_split(
                    mdl[mdl.Bond==b][vars_].to_numpy(),
                    mdl[mdl.Bond==b].FCY,test_size=0.25, random_state=20)

    y_prob = lr.predict(X_test)
    y_pred = [0 if p<0.5 else 1 for p in y_prob]

    # model accuracy versus guess major class
    accuracy.append(metrics.accuracy_score(y_test.values, y_pred))

df_results.loc['Observations'] = [len(mdl[mdl.Bond==b]) for b in [0,1]]
df_results.loc['McFaddens R-squared'] = \
                ['{:.2f}'.format(log_reg[i].prsquared) 
                for i in log_reg.keys()]

df_results.loc['Accuracy'] = \
                ['{:.1f}'.format(100*a) for a in accuracy]

df_results=df_results.rename(columns={i:'Reg {}'.format(i+1)
                          for i in log_reg.keys()})

np.set_printoptions(linewidth=110)
df_results=df_results.rename(index= {'FX_short':'FX_RISK_SHORT',
                                     'FX_long':'FX_RISK_LONG',
                           'Ratings FC': 'SOV_RISK',
                           'Pol_Risk':'POL_RISK',
                           'Amount':'AMOUNT',
                           'Domestic':'DOMESTIC',
                           'Bond':'BOND',
                           'GDP Growth': 'GDP_GROWTH',
                           'Tenor':'TENOR'})
df_results.transpose()

Optimization terminated successfully.
         Current function value: 0.390795
         Iterations: 92
         Function evaluations: 95
         Gradient evaluations: 95
Optimization terminated successfully.
         Current function value: 0.029906
         Iterations: 183
         Function evaluations: 189
         Gradient evaluations: 189


,FX_RISK_SHORT,SOV_RISK,POL_RISK,VIX,AMOUNT,DOMESTIC,GDP_GROWTH,TENOR,Observations,McFaddens R-squared,Accuracy
Reg 1,66.86***(15.39),0.14 (0.09),0.80 (0.45),0.04*(0.02),0.44***(0.08),-3.32***(0.77),0.01 (0.03),-0.12***(0.02),741,0.38,81.7
Reg 2,336.41*(158.16),5.89*(2.71),3.90 (9.03),0.31*(0.13),10.62*(4.25),-64.73*(28.24),0.43 (0.32),0.20 (0.14),371,0.94,98.9


In [129]:
df_results.to_csv('./../results/Table_S_bonds_vs_loans.csv')

<h2 style='text-align: center;'> Supplementary Tenor Fixed Effects </h2>

In [144]:
countries = ['India',
             'Mexico',
             'Brazil',
             'South Africa',
             'Indonesia',
             'Thailand',
            # 'Malaysia',
            # 'Turkey'
            ]

In [145]:
mdl = data.copy();
mdl = mdl[mdl.Country.apply(lambda x: x in countries)]
len(mdl)

1112

In [146]:
mdl.keys()

Index(['Quarter', 'Year', 'Country', 'Delta_Interest_EU', 'Delta_Interest_JPY',
       'Delta_Interest_US', 'ECB', 'Exchange_rate', 'FFR', 'FX_long',
       'FX_short', 'Fin_depth', 'Fin_dev', 'GDP Growth', 'GDP per cap',
       'Interest rate', 'JPY', 'Pol_Risk', 'Ratings FC', 'Ratings LC', 'VIX',
       'WUI', 'market_volume_bonds', 'market_volume_loans', 'Amount', 'LCY',
       'Currency', 'Domestic', 'Corporate', 'Industry', 'Bond', 'Tenor',
       'Tenor_bracket', 'FCY'],
      dtype='object')

In [147]:
# Create a DataFrame with the one-hot encoded columns

# Drop first column to avoid the dummy variable trap
# ------ One-Hot Country encoding --------------- #
one_hot_df = pd.get_dummies(mdl.Country, drop_first=True)
mdl = pd.concat([mdl, one_hot_df], axis=1)
FE_country_ = ['India',
              'Mexico',
              'South Africa',
              'Indonesia',
              'Thailand',
               #'Malaysia',
              # 'Turkey'
              ]

# ------ One-Hot Tenor encoding --------------- #
one_hot_df = pd.get_dummies(mdl.Tenor_bracket, drop_first = True)
one_hot_df=one_hot_df.rename(columns = {'0-5 yrs':'tenor_1',
                    '5-10 yrs': 'tenor_2',
                    '>10 yrs': 'tenor_3',
                             } )
mdl = pd.concat([mdl, one_hot_df], axis=1)
FE_tenor_ = ['tenor_{}'.format(i) for i in [2,3]] 

#-------- TRANSFORM VARIABLES ------------#
mdl['Amount'] = mdl['Amount'].apply(lambda x: np.log(x))
mdl['WUI'] = mdl['WUI'].apply(lambda x: x/1000)

In [152]:
### Table for main
vars_cntrl_0 = ['Amount','Domestic','Bond',
              'GDP Growth','GDP per cap', 'Tenor']

vars_cntrl_1 = ['Amount','Domestic','Bond',
              'GDP Growth','GDP per cap']

vars_list_ = [
        ['FX_short','Ratings FC','Pol_Risk','VIX'] + vars_cntrl_0 + FE_country_,
        ['FX_short','Ratings FC','Pol_Risk','VIX'] + vars_cntrl_1 + FE_country_ + FE_tenor_,
        ]

all_vars_ = ['FX_short','Ratings FC','Pol_Risk','VIX','Tenor'] + \
             vars_cntrl_

log_reg = {}
for i,vars_ in enumerate(vars_list_):
    log_reg[i] = \
     sm.Logit(mdl.FCY, mdl[vars_].to_numpy()).fit(maxiter=1000)
    
df_results = pd.DataFrame(
                columns=[i for i,v in enumerate(vars_list_)],
                index = all_vars_
                        )

for i, vars_ in enumerate(vars_list_):
    for j,v in enumerate(vars_):
        if v in all_vars_:
            df_results.loc[v][i] = '{:.2f}{}({:.2f})'.format(
                            log_reg[i].params[j],
                            getSig(log_reg[i].pvalues[j]),
                            #log_reg[i].pvalues[j],
                            log_reg[i].bse[j])

accuracy = []
for i,lr in log_reg.items():
    X_train, X_test, y_train, y_test = train_test_split(
                    mdl[vars_list_[i]],
                    mdl.FCY,test_size=0.33, random_state=20)

    y_prob = lr.predict(X_test)
    y_pred = [0 if p<0.5 else 1 for p in y_prob]

    # model accuracy versus guess major class
    accuracy.append(metrics.accuracy_score(y_test.values, y_pred))

df_results.loc['Observations'] = [len(mdl)]*len(vars_list_)
df_results.loc['McFaddens R-squared'] = \
                ['{:.2f}'.format(log_reg[i].prsquared) 
                for i in log_reg.keys()]

df_results.loc['Accuracy'] = \
                ['{:.1f}'.format(100*a) for a in accuracy]

df_results=df_results.rename(columns={i:'Reg {}'.format(i+1)
                          for i in log_reg.keys()})

np.set_printoptions(linewidth=110)
df_results=df_results.rename(index= {'FX_short':'FX_RISK',
                           'Ratings FC': 'SOV_RISK',
                           'Pol_Risk':'POL_RISK',
                           'Amount':'AMOUNT',
                           'Domestic':'DOMESTIC',
                           'Bond':'BOND',
                           'GDP Growth': 'GDP_GROWTH',
                           'Tenor':'TENOR'})
df_results.transpose()

Optimization terminated successfully.
         Current function value: 0.351882
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.342786
         Iterations 8


,FX_RISK,SOV_RISK,POL_RISK,VIX,TENOR,AMOUNT,DOMESTIC,BOND,GDP_GROWTH,GDP per cap,Observations,McFaddens R-squared,Accuracy
Reg 1,76.50***(12.72),0.40***(0.11),0.17 (0.41),0.05***(0.01),-0.09***(0.02),0.69***(0.07),-5.30***(0.69),-1.87***(0.26),0.04 (0.03),-0.00 (0.00),1112,0.42,85.3
Reg 2,75.75***(13.09),0.41***(0.11),-0.00 (0.42),0.05***(0.01),NaN,0.66***(0.08),-5.16***(0.68),-1.91***(0.26),0.04 (0.03),-0.00 (0.00),1112,0.43,85.3


In [154]:
df_results.to_csv('./../results/Table_S_tenor_FE.csv')

<h2 style='text-align: center;'> Supplementary Table: Interest rate diff </h2>

In [29]:
countries = ['India',
             'Mexico',
             'Brazil',
             'South Africa',
             'Indonesia',
             'Thailand',
            # 'Malaysia',
            # 'Turkey'
            ]

In [30]:
mdl = data.copy();
mdl = mdl[mdl.Country.apply(lambda x: x in countries)]
#mdl = mdl[(mdl.LCY==1) | (mdl.Currency == 'USD')]
len(mdl)

1112

In [31]:
mdl.keys()

Index(['Quarter', 'Year', 'Country', 'Delta_Interest_EU', 'Delta_Interest_JPY',
       'Delta_Interest_US', 'ECB', 'Exchange_rate', 'FFR', 'FX_long',
       'FX_short', 'Fin_depth', 'Fin_dev', 'GDP Growth', 'GDP per cap',
       'Interest rate', 'JPY', 'Pol_Risk', 'Ratings FC', 'Ratings LC', 'VIX',
       'WUI', 'market_volume_bonds', 'market_volume_loans', 'Amount', 'LCY',
       'Currency', 'Domestic', 'Corporate', 'Industry', 'Bond', 'Tenor',
       'Tenor_bracket', 'FCY'],
      dtype='object')

In [32]:
# Create a DataFrame with the one-hot encoded columns

# Drop first column to avoid the dummy variable trap
# ------ One-Hot Country encoding --------------- #
one_hot_df = pd.get_dummies(mdl.Country, drop_first=True)
mdl = pd.concat([mdl, one_hot_df], axis=1)
FE_country_ = ['India',
              'Mexico',
              'South Africa',
              'Indonesia',
              'Thailand',
               #'Malaysia',
              # 'Turkey'
              ]

# ------ One-Hot Tenor encoding --------------- #
one_hot_df = pd.get_dummies(mdl.Tenor_bracket, drop_first = True)
one_hot_df=one_hot_df.rename(columns = {'0-5 yrs':'tenor_1',
                    '5-10 yrs': 'tenor_2',
                    '>10 yrs': 'tenor_3',
                             } )
mdl = pd.concat([mdl, one_hot_df], axis=1)
FE_tenor_ = ['tenor_{}'.format(i) for i in [2,3]] 

#-------- TRANSFORM VARIABLES ------------#
mdl['Amount'] = mdl['Amount'].apply(lambda x: np.log(x))
mdl['WUI'] = mdl['WUI'].apply(lambda x: x/1000)

In [33]:
### Table for main
vars_cntrl = ['Amount','Domestic','Bond',
              'GDP Growth','GDP per cap', 'Tenor']

vars_list_ = [
        ['FX_short','Ratings FC','Pol_Risk','VIX','Delta_Interest_US'] + vars_cntrl_,
        ['FX_short','Ratings FC','Pol_Risk','VIX'] + vars_cntrl_
        ]

all_vars_ = ['FX_short','Ratings FC','Pol_Risk','VIX','Delta_Interest_US'] + \
             vars_cntrl_

log_reg = {}
for i,vars_ in enumerate(vars_list_):
    log_reg[i] = \
     sm.Logit(mdl.FCY, mdl[vars_].to_numpy()).fit(maxiter=1000)
    
df_results = pd.DataFrame(
                columns=[i for i,v in enumerate(vars_list_)],
                index = all_vars_
                        )

for i, vars_ in enumerate(vars_list_):
    for j,v in enumerate(vars_):
        if v in all_vars_:
            df_results.loc[v][i] = '{:.2f}{}({:.2f})'.format(
                            log_reg[i].params[j],
                            getSig(log_reg[i].pvalues[j]),
                            #log_reg[i].pvalues[j],
                            log_reg[i].bse[j])

accuracy = []
for i,lr in log_reg.items():
    X_train, X_test, y_train, y_test = train_test_split(
                    mdl[vars_list_[i]],
                    mdl.FCY,test_size=0.33, random_state=20)

    y_prob = lr.predict(X_test)
    y_pred = [0 if p<0.5 else 1 for p in y_prob]

    # model accuracy versus guess major class
    accuracy.append(metrics.accuracy_score(y_test.values, y_pred))

df_results.loc['Observations'] = [len(mdl)]*len(vars_list_)
df_results.loc['McFaddens R-squared'] = \
                ['{:.2f}'.format(log_reg[i].prsquared) 
                for i in log_reg.keys()]

df_results.loc['Accuracy'] = \
                ['{:.1f}'.format(100*a) for a in accuracy]

df_results=df_results.rename(columns={i:'Reg {}'.format(i+1)
                          for i in log_reg.keys()})

np.set_printoptions(linewidth=110)
df_results=df_results.rename(index= {'FX_short':'FX_RISK',
                           'Ratings FC': 'SOV_RISK',
                           'Pol_Risk':'POL_RISK',
                           'Amount':'AMOUNT',
                           'Domestic':'DOMESTIC',
                           'Bond':'BOND',
                           'GDP Growth': 'GDP_GROWTH',
                           'Tenor':'TENOR'})
df_results.transpose()

Optimization terminated successfully.
         Current function value: 0.389532
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.389842
         Iterations 8


,FX_RISK,SOV_RISK,POL_RISK,VIX,Delta_Interest_US,AMOUNT,DOMESTIC,BOND,GDP_GROWTH,TENOR,GDP per cap,Observations,McFaddens R-squared,Accuracy
Reg 1,67.51***(9.71),0.55***(0.07),0.33 (0.34),0.04***(0.01),0.03 (0.03),0.67***(0.07),-5.81***(0.77),-1.73***(0.24),0.07*(0.03),-0.10***(0.02),-0.00***(0.00),1112,0.35,83.7
Reg 2,69.08***(9.54),0.53***(0.06),0.25 (0.32),0.04***(0.01),NaN,0.68***(0.07),-5.66***(0.73),-1.76***(0.23),0.07**(0.03),-0.10***(0.01),-0.00***(0.00),1112,0.35,83.7


In [25]:
len(mdl)/len(data)

0.6981747066492829

In [28]:
len(data[data.Currency=='USD'])/len(data[data.FCY==1])

0.8260869565217391

In [38]:
[(c,100*len(data[data.Currency==c])/len(data[data.FCY==1])) for c in 
['USD', 'EUR', 'GBP', 'JPY', 'SGD', 'NOK', 'AUD', 'SAR']]

[('USD', 82.6086956521739),
 ('EUR', 13.285024154589372),
 ('GBP', 0.24154589371980675),
 ('JPY', 1.4492753623188406),
 ('SGD', 0.7246376811594203),
 ('NOK', 0.966183574879227),
 ('AUD', 0.4830917874396135),
 ('SAR', 0.24154589371980675)]

In [36]:
data[data.FCY==1].Currency.unique()

array(['USD', 'EUR', 'GBP', 'JPY', 'SGD', 'NOK', 'AUD', 'SAR'], dtype=object)